# SQL query from table names - Continued

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [6]:
context = [ {'role':'system', 'content':"""
    CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    hire_date DATE,
    department_id INTEGER);
             
    CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT);
             
    CREATE TABLE salaries (
    employee_id INTEGER PRIMARY KEY,
    salary REAL,
    effective_date DATE);
"""} ]



In [7]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
    user query: What employee is paid the most?
    SQL query:
        SELECT 
            employee_id, first_name, last_name
        FROM 
            employees e
        JOIN
            salaries s
        ON
            e.employee_id == s.employee_id
        ORDER BY
            salary DESC
        LIMIT 1;
"""
})

In [8]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [9]:
#new
context_user = context.copy()
print(return_CCRMSQL("""what employee has the highest salary""", context_user))

SELECT 
    employee_id, first_name, last_name
FROM 
    employees e
JOIN
    salaries s
ON
    e.employee_id == s.employee_id
ORDER BY
    salary DESC
LIMIT 1;


In [10]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("what employee has the highest salary", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query selects the name of the employee who has the highest salary by joining the "employees" table with the "salary" table on the ID_usr field. It then orders the result by salary in descending order and limits the output to only one row, which represents the employee with the highest salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [13]:
#new
context_user = context.copy()
print(return_CCRMSQL("""Show me the last five hires, order them by salary, having the highest paid at top""", context_user))

```sql
SELECT 
    e.employee_id, e.first_name, e.last_name, s.salary
FROM 
    employees e
JOIN
    salaries s
ON
    e.employee_id = s.employee_id
ORDER BY
    hire_date DESC, salary DESC
LIMIT 5;
```


In [14]:
#new
context_user = context.copy()
print(return_CCRMSQL("""show me how many people work in each department and how much salary we are paying for each department""", context_user))

```sql
SELECT 
    d.department_name,
    COUNT(e.employee_id) AS num_employees,
    SUM(s.salary) AS total_salary
FROM 
    departments d
JOIN 
    employees e ON d.department_id = e.department_id
JOIN 
    salaries s ON e.employee_id = s.employee_id
GROUP BY 
    d.department_name;
```


In [16]:
#new
context_user = context.copy()
print(return_CCRMSQL("""add to every employee the standarad deviation for their salary regarding to their department""", context_user))

To calculate the standard deviation of salaries for each employee's department and add it to the result, you can use the following SQL query:

```sql
WITH department_salary_stats AS (
    SELECT
        d.department_id,
        AVG(s.salary) AS avg_salary,
        GROUP_CONCAT(s.salary) AS salaries
    FROM
        departments d
    JOIN
        employees e ON d.department_id = e.department_id
    JOIN
        salaries s ON e.employee_id = s.employee_id
    GROUP BY
        d.department_id
),
employee_salary_stats AS (
    SELECT
        e.employee_id,
        e.first_name,
        e.last_name,
        e.department_id,
        s.salary,
        d.avg_salary,
        department_salary_stats.salaries
    FROM
        employees e
    JOIN
        salaries s ON e.employee_id = s.employee_id
    JOIN
        department_salary_stats ON e.department_id = department_salary_stats.department_id
),
employee_salary_stats_with_stddev AS (
    SELECT
        employee_id,
        first_name,
        

The one shot approach is very successful, all queries seem to be fulfilled perfectly, even hard cases like the last one are fulfilled in a few seconds